# Transfer Learning with RAG Models
----
**Objective**: In this notebook, you will experiment with a QA RAG model on the climate_fever dataset using the Haystack framework. You will get to go through the whole process of fitting the parts of a RAG model together, and learn how to prompt it with queries to get answers from the provided dataset.

NOTE: Make sure to change the runtime from CPU to TPU or GPU for faster training

## Install Libraries
Install the Haystack (for colab) and Datasets libraries

In [5]:
!pip install farm-haystack[colab]
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 819.7/819.7 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.7/75.7 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 52.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.7/48.7 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 68.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 67.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 65.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 66.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 27.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 48.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 51.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.8/50.8 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.9 MB/s eta 0:00:00


## Import Dataset
----
In this section, we will use as an example the climate_fever dataset. The dataset consists of 1535 rows of claims about climate change, and they either refute or support climate change, with some claims being neutral. We will build with a specific topic in mind so that we can get more accurate answers, and keep in mind that bigger datasets with open topics can also be used.

**Question 1**: Use the "load_dataset" function to load the "climate_fever" dataset with the "test" split.

In [6]:
from datasets import load_dataset

# Load the dataset
data = load_dataset("climate_fever", split="test")


Generating test split:   0%|          | 0/1535 [00:00<?, ? examples/s]

## Formatting and Writing Documents
----
First, we need to extract, format and write the documents from our chosen dataset so that we can later build our QA Pipeline. This Pipeline will facilitate the process of building our RAG model and getting answers from it.

Keep in mind that for this notebook we will focus on how to build the pipeline with the simplest configurations. Feel free to experiment with different parameters.



In [7]:
data[0]

{'claim_id': '0',
 'claim': 'Global warming is driving polar bears toward extinction',
 'claim_label': 0,
 'evidences': [{'evidence_id': 'Extinction risk from global warming:170',
   'evidence_label': 2,
   'article': 'Extinction risk from global warming',
   'evidence': '"Recent Research Shows Human Activity Driving Earth Towards Global Extinction Event".',
   'entropy': 0.6931471824645996,
   'votes': ['SUPPORTS', 'NOT_ENOUGH_INFO', None, None, None]},
  {'evidence_id': 'Global warming:14',
   'evidence_label': 0,
   'article': 'Global warming',
   'evidence': 'Environmental impacts include the extinction or relocation of many species as their ecosystems change, most immediately the environments of coral reefs, mountains, and the Arctic.',
   'entropy': 0.0,
   'votes': ['SUPPORTS', 'SUPPORTS', None, None, None]},
  {'evidence_id': 'Global warming:178',
   'evidence_label': 2,
   'article': 'Global warming',
   'evidence': 'Rising temperatures push bees to their physiological limits,

**Question 2**: Use the write_documents method to save the formatted documents into document_storage

In [8]:
!pip install --upgrade tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 489.8/489.8 MB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 48.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.7/440.7 kB 30.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 47.5 MB/s eta 0:00:00
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.12.0
    Uninstalling tensorflow-estimator-2.12.0:
      Successfully uninstalled tensorflow-estimator-2.12.0
  Attempting uninstall: keras
    Found existing installation: keras 2.12.0
    Uninstalling keras-2.12.0:
      Successfully uninstalled keras-2.12.0
  Attempting uninstall: google-auth-oauthlib
    Found existing installation: google-auth-oauthlib 0.4.6
    Uninstalling google-auth-oauthlib-0.4.6:
      Successfully uninstalled google-auth-oauthlib-0.4.6
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.12.0
    Uninstalling te

In [9]:
from haystack.document_stores import InMemoryDocumentStore

# Create an InMemoryDocumentStore
document_storage = InMemoryDocumentStore(use_bm25=True)

# Extract and format the documents from the dataset

documents = []
for example in data:
    claim_id = example["claim_id"]
    text = example["claim"]
    content = " ".join([ev["evidence"] for ev in example["evidences"]])  # Combine evidence texts
    document = {"text": text, "content": content, "meta": {"claim_id": claim_id}}
    documents.append(document)


# Write the documents to the document store

document_storage.write_documents(documents)

Updating BM25 representation...: 100%|██████████| 1525/1525 [00:00<00:00, 6905.75 docs/s]


In [29]:
documents2 = [{"content": document['claim']} for document in data]

In [30]:
documents2

[{'content': 'Global warming is driving polar bears toward extinction'},
 {'content': 'The sun has gone into ‘lockdown’ which could cause freezing weather, earthquakes and famine, say scientists'},
 {'content': 'The polar bear population has been growing.'},
 {'content': "Ironic' study finds more CO2 has slightly cooled the planet"},
 {'content': 'Human additions of CO2 are in the margin of error of current measurements and the gradual increase in CO2 is mainly from oceans degassing as the planet slowly emerges from the last ice age.'},
 {'content': 'They tell us that we are the primary forces controlling earth temperatures by the burning of fossil fuels and releasing their carbon dioxide.'},
 {'content': 'The Great Barrier Reef is experiencing the most widespread bleaching ever recorded'},
 {'content': 'it’s not a pollutant that threatens human civilization.'},
 {'content': 'If CO2 was so terrible for the planet, then installing a CO2 generator in a greenhouse would kill the plants.'}

In [10]:
documents[0]

{'text': 'Global warming is driving polar bears toward extinction',
 'content': '"Recent Research Shows Human Activity Driving Earth Towards Global Extinction Event". Environmental impacts include the extinction or relocation of many species as their ecosystems change, most immediately the environments of coral reefs, mountains, and the Arctic. Rising temperatures push bees to their physiological limits, and could cause the extinction of bee populations. Rising global temperatures, caused by the greenhouse effect, contribute to habitat destruction, endangering various species, such as the polar bear. "Bear hunting caught in global warming debate".',
 'meta': {'claim_id': '0'}}

In [11]:
documents[1]

{'text': 'The sun has gone into ‘lockdown’ which could cause freezing weather, earthquakes and famine, say scientists',
 'content': "The current consensus of the scientific community is that the aerosols and dust released into the upper atmosphere causes cooler temperatures by preventing the sun's energy from reaching the ground. The Little Ice Age caused crop failures and famines in Europe. The persistently cold, wet weather caused great hardship, was primarily responsible for the Great Famine of 1315–1317, and strongly contributed to the weakened immunity and malnutrition leading up to the Black Death (1348–1350). The manifestation of the meteorological winter (freezing temperatures) in the northerly snow–prone latitudes is highly variable depending on elevation, position versus marine winds and the amount of precipitation. In many regions, winter is associated with snow and freezing temperatures.",
 'meta': {'claim_id': '5'}}

## Preparing the Retriever
----
We need to prepare our Retriever node of our pipeline. It will be responsible to get the documents from our document storage, so that they can be used by the Language Model later. We will the BM25Retriever provided by haystack, as it is the recommended Retriever for begginners.

**Question 3**: Create the BM25Retriever using the document_storage created earlier, with a top_k of value 2

In [12]:
from haystack.nodes import BM25Retriever

# Note: The higher the top_k is, the better the answer will be. However, speed will be affected
retriever = BM25Retriever(document_store=document_storage, top_k=2)


## Preparing the Language Model
----
Now, we will prepare our Language Model using the prompt node. We need to first create our prompt, and for that, Haystack requires a specific structure. We will then define our desired language model alongside the prompt template we created. When creating this template, we need to Parse the output to a format that Haystack can use.

**Question 4**: Define the prompt node using PromptNode with the model name as "google/flan-t5-large" and the default prompt template as the created "rag_prompt"

In [31]:
from haystack.nodes.prompt import prompt_template
from haystack.nodes import PromptNode, PromptTemplate, AnswerParser

rag_prompt = PromptTemplate(
    prompt="""Create comprehensive answers from the related text given the questions.
                             Provide a clear and concise response that displays the key points and information presented in the related text.
                             Your answer should be in your own words and be no longer than 50 words.
                             \n\n Related text: {join(documents)} \n\n Question: {query} \n\n Answer:""",
    output_parser=AnswerParser(),
)

prompt_node = PromptNode(model_name_or_path="google/flan-t5-large", default_prompt_template= rag_prompt)




In [14]:
prompt_node = PromptNode(default_prompt_template= rag_prompt)

(…)le/flan-t5-base/resolve/main/config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

(…)base/resolve/main/generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

(…)-base/resolve/main/tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

(…)flan-t5-base/resolve/main/tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

(…)ase/resolve/main/special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

## Fitting our Pipeline Together
----
Finally, we are going to put our pipeline nodes together. For that we will use the Pipeline function from haystack. With the pipeline ready you will be able to ask it questions and get answers

**Question 5**: Add the retriever node and prompt_node created in the previous steps to the Pipeline using the add_node function. Hint: you need to provide the inputs to each of these nodes.

In [15]:
from haystack.pipelines import Pipeline

pipe = Pipeline()



In [24]:
# Add the retriever node, specifying the name and inputs
pipe.add_node(retriever, name="retriever", inputs=["Query"])

# Add the prompt node, specifying the name and inputs
pipe.add_node(prompt_node, name="prompt_node", inputs=["retriever"])


## Asking the RAG Model Questions
----
We use the pipeline .run() method to ask a question. Since the output provided by our Prompt Node is a Haystack object, we retrieve in the way provided inside the print() function.

In [25]:
output = pipe.run(query="When did global warming start")

print(output["answers"][0].answer)

1993.


In [28]:
output = pipe.run(query="What is the biggest damaging factor for the climate?")

print(output["answers"][0].answer)

30% in 2010[update].


In [ ]:
output = pipe.run(query="Who is most responsible for pollution")

print(output["answers"][0].answer)

In [ ]:
# Here are some other examples you can use
examples = [
    "Who is most responsible for pollution",
    "What is the biggest damaging factor for the climate?",
    "What are some clean energy sources?",
    "How much does the average temperature of our planet rise per decade?"
]